In [110]:
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.cross_validation import cross_val_score
from sklearn.decomposition import PCA
from sklearn import grid_search
from sklearn import preprocessing
from sklearn.svm import SVC
from datetime import datetime
import xgboost as xgb

# %matplotlib inline

In [111]:
#summary statistics of users' age group, gender, country of destination
#extract age of each country_destination as it simliar age gorup people tend to go to places with simliar age groups?
# summary statistics of destination countries in this dataset and their locations
#training dataset
train_df = pd.read_csv("./data/train_users_2.csv")
#testing dataset
test_df = pd.read_csv("./data/test_users.csv")
#session data
session_test = pd.read_csv("./data/session_test.csv")
session_train = pd.read_csv("./data/session_train.csv")

In [112]:
#combine session_test with session_train
session_df = session_train.append(session_test,ignore_index=True)

In [113]:
#rename
session_df.rename(columns={'user_id':'id'}, inplace=True)


In [114]:
session_secs_elapsed = session_df.groupby('id')
session_secs_elapsed = session_secs_elapsed.agg({'secs_elapsed' : 'sum'})
session_secs_elapsed.reset_index(inplace=True)


In [115]:
ids = session_df.id
session_df.drop('secs_elapsed',inplace=True,axis=1)

In [116]:
items_counts = session_df['action'].value_counts()
top_100 = items_counts[0:100].index

In [117]:
print(session_df[session_df.action.isin(top_100)].shape)
print(session_df.shape)

(10262152, 2)
(10397758, 2)


In [118]:
sub_session_df = session_df[session_df.action.isin(top_100)]

In [119]:
sub_session_df.reset_index(inplace=True,drop=True)

In [120]:
sub_session_df.head()

,id,action
0,d1mm9tcy42,lookup
1,d1mm9tcy42,search_results
2,d1mm9tcy42,lookup
3,d1mm9tcy42,search_results
4,d1mm9tcy42,lookup


In [121]:
dummies = pd.get_dummies(sub_session_df.action, prefix="d")

In [122]:
# normalizer = preprocessing.Normalizer().fit(dummies)
# dummies = normalizer.transform(dummies)  

In [123]:
# dummies = normalizer.transform(dummies) 

In [124]:
dummies = pd.concat((sub_session_df, pd.DataFrame(dummies)), axis=1)

In [125]:
dummies.drop(['action'],axis=1,inplace=True)
dummies.head()

,id,d_account,d_active,d_agree_terms_check,d_ajax_check_dates,d_ajax_get_referrals_amt,d_ajax_image_upload,d_ajax_lwlb_contact,d_ajax_photo_widget_form_iframe,d_ajax_referral_banner_experiment_type,...,d_similar_listings,d_similar_listings_v2,d_social_connections,d_status,d_track_page_view,d_travel_plans_current,d_unavailabilities,d_update,d_verify,d_view
0,d1mm9tcy42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,d1mm9tcy42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,d1mm9tcy42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,d1mm9tcy42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,d1mm9tcy42,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
dummies = dummies.groupby('id')
dummies = dummies.agg(np.sum)


In [128]:
dummies

,d_account,d_active,d_agree_terms_check,d_ajax_check_dates,d_ajax_get_referrals_amt,d_ajax_image_upload,d_ajax_lwlb_contact,d_ajax_photo_widget_form_iframe,d_ajax_referral_banner_experiment_type,d_ajax_referral_banner_type,...,d_similar_listings,d_similar_listings_v2,d_social_connections,d_status,d_track_page_view,d_travel_plans_current,d_unavailabilities,d_update,d_verify,d_view
id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,2,0,0,0,0
0010k6l0om,0,0,0,0,0,0,0,0,0,0,...,0,8,0,0,0,0,0,0,0,0
001wyh0pz8,0,4,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,1,0,0
0028jgx1x1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
002qnbzfs5,0,29,0,0,0,0,0,0,0,0,...,0,0,72,0,0,0,21,13,0,0
0031awlkjq,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0035hobuyj,0,0,0,2,0,0,2,0,0,0,...,12,0,0,0,0,0,0,5,0,0
00378ocvlh,0,0,0,1,0,6,1,0,0,0,...,0,2,0,0,0,0,0,4,0,0
00389675gq,1,3,0,2,0,0,2,0,0,0,...,17,0,1,0,0,0,0,1,1,0


In [129]:
dummies = dummies.reset_index()

In [130]:
sessions_feat = session_secs_elapsed.merge(dummies, on='id', how='left')
sessions_feat.fillna(-1,inplace=True)
sessions_feat.id.unique()

In [132]:
sessions_feat.id.unique()

array(['00023iyk9l', '0010k6l0om', '001wyh0pz8', ..., 'zzysuoqg6x',
       'zzywmcn0jv', 'zzzlylp57e'], dtype=object)

## training data

In [133]:
train_with_session = train_df[train_df.id.isin(sessions_feat.id.unique())].reset_index(drop=True)
train_without_session = train_df[~train_df.id.isin(sessions_feat.id.unique())].reset_index(drop=True)
##
sample_index = np.random.choice(train_without_session.shape[0], 50000)

In [134]:
print(train_with_session.shape)
print(train_without_session.iloc[sample_index].shape)

(72631, 16)
(50000, 16)


In [135]:
training_sample = pd.concat([train_with_session, train_without_session.iloc[sample_index]]).reset_index(drop=True)


In [136]:
# labels = train_df.country_destination
train_id = training_sample['id']
# train_features= train_df
features = training_sample.drop(['country_destination'],axis=1)
labels = training_sample['country_destination']
# features = train_df
test_id = test_df['id']

In [137]:
#combine train and test data for
features = pd.concat([features, test_df])
features.reset_index(inplace=True,drop=True)

In [138]:
#combine features with sessions
features = pd.merge(features, sessions_feat, on="id",how="left")
# test_features = pd.merge(test_df,grouped,on="id",how="left")

In [139]:
features.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,d_similar_listings,d_similar_listings_v2,d_social_connections,d_status,d_track_page_view,d_travel_plans_current,d_unavailabilities,d_update,d_verify,d_view
0,d1mm9tcy42,2014-01-01,20140101000936,2014-01-04,MALE,62,basic,0,en,sem-non-brand,...,9,0,0,0,0,0,0,0,0,0
1,yo8nz8bqcq,2014-01-01,20140101001558,NaN,-unknown-,NaN,basic,0,en,direct,...,0,0,0,0,0,0,0,0,0,0
2,4grx6yxeby,2014-01-01,20140101001639,NaN,-unknown-,NaN,basic,0,en,sem-brand,...,0,0,0,0,0,0,0,0,1,0
3,ncf87guaf0,2014-01-01,20140101002146,NaN,-unknown-,NaN,basic,0,en,direct,...,28,0,0,0,6,0,0,2,0,0
4,4rvqpxoh3h,2014-01-01,20140101002619,2014-01-02,-unknown-,NaN,basic,25,en,direct,...,0,0,0,0,0,0,1,0,0,0


In [140]:

#util data

age_device_averages = pd.pivot_table(features[pd.notnull(features.age)], values='age', index=['first_browser'], aggfunc=np.mean)

In [141]:
#util funciton to impute gender
def prepare_gender(row):
    if row.gender == "-unknown-":
        return 'OTHER'
    elif pd.isnull(row.gender):
        return 'OTHER'
    else:
        return row.gender
    
#util grab month date_account was created
def prepare_date_created(row):
    dateobj = datetime.strptime(row['date_account_created'], '%Y-%m-%d')
    return dateobj
def prepare_date_active(row):
    dateobj =datetime.strptime(str(row['timestamp_first_active']), '%Y%m%d%H%M%S')
    return dateobj

def clean_age(row):
    return_val = int()
    if row['age'] >= 2000:
        return_val = None
    elif row['age'] > 1800:
        return_val = 2015 - row['age']
    elif row['age'] > 120:
        return_val = None
    else:
        return_val = row['age']
    if return_val <= 15:
        return None
    elif return_val >= 100:
        return None
    else:
        return return_val



def impute_age(row):
    return_val = int()
    if pd.isnull(row.age):
        if row.first_browser in age_device_averages:
            return_val = age_device_averages[row.first_browser]
        else:
            return_val = features.age.mean()
    else:
        return_val = row.age
    return return_val
    

# def pepare_year_created(row):
#     dateobj = datetime.strptime(row['date_account_created'], '%Y-%m-%d')
#     return date.year

# def prepare_month_first_active(row):
#     dateobj =datetime.strptime(str(train_df.iloc[0]['timestamp_first_active']), '%Y%m%d%H%M%S')
#     return date.month

# def prepare_year_first_active(row)
    

    


In [142]:
def prep_df(df):
    #drop date_first booking as it will not be in test data set
    dropcol = ['date_first_booking']
    df.drop(dropcol,axis=1, inplace=True)
    #convert the dates
    df['date_account_created'] = df.apply(lambda x: prepare_date_created(x),axis=1)
    df['timestamp_first_active'] = df.apply(lambda x: prepare_date_active(x),axis=1)
    #convert gender
    df['gender'] = df.apply(lambda x : prepare_gender(x), axis=1)
    #fix age
    df['age'] = df.apply(lambda x : clean_age(x), axis=1)
    df['secs_elapsed'].fillna(-1,inplace=True)
#     df.fillna(-1,inplace=True)

def engineer_feat(df):
    df['dac_year'] = df['date_account_created'].apply(lambda x: x.year)
    df['dac_month'] = df['date_account_created'].apply(lambda x: x.month)
    df['dac_day'] = df['date_account_created'].apply(lambda x: x.day)
    
    df['tfc_year'] = df['timestamp_first_active'].apply(lambda x: x.year)
    df['tfc_month'] = df['timestamp_first_active'].apply(lambda x: x.month)
    df['tfc_day'] = df['timestamp_first_active'].apply(lambda x: x.day)
    
    #drop date account_created and timestamp_first_active
    dropcol = ['date_account_created','timestamp_first_active']
    df.drop(dropcol,axis=1,inplace=True)
    
    

In [143]:
prep_df(features)
age_device_averages = pd.pivot_table(features[pd.notnull(features.age)], values='age', index=['first_browser'], aggfunc=np.mean)
features['age'] = features.apply(lambda x : impute_age(x), axis=1)
features.head()

,id,date_account_created,timestamp_first_active,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,...,d_similar_listings,d_similar_listings_v2,d_social_connections,d_status,d_track_page_view,d_travel_plans_current,d_unavailabilities,d_update,d_verify,d_view
0,d1mm9tcy42,2014-01-01,2014-01-01 00:09:36,MALE,62.000000,basic,0,en,sem-non-brand,google,...,9,0,0,0,0,0,0,0,0,0
1,yo8nz8bqcq,2014-01-01,2014-01-01 00:15:58,OTHER,37.861191,basic,0,en,direct,direct,...,0,0,0,0,0,0,0,0,0,0
2,4grx6yxeby,2014-01-01,2014-01-01 00:16:39,OTHER,37.861191,basic,0,en,sem-brand,google,...,0,0,0,0,0,0,0,0,1,0
3,ncf87guaf0,2014-01-01,2014-01-01 00:21:46,OTHER,33.590916,basic,0,en,direct,direct,...,28,0,0,0,6,0,0,2,0,0
4,4rvqpxoh3h,2014-01-01,2014-01-01 00:26:19,OTHER,33.001512,basic,25,en,direct,direct,...,0,0,0,0,0,0,1,0,0,0


In [144]:
engineer_feat(features)
features.head()

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,...,d_unavailabilities,d_update,d_verify,d_view,dac_year,dac_month,dac_day,tfc_year,tfc_month,tfc_day
0,d1mm9tcy42,MALE,62.000000,basic,0,en,sem-non-brand,google,omg,Web,...,0,0,0,0,2014,1,1,2014,1,1
1,yo8nz8bqcq,OTHER,37.861191,basic,0,en,direct,direct,untracked,Web,...,0,0,0,0,2014,1,1,2014,1,1
2,4grx6yxeby,OTHER,37.861191,basic,0,en,sem-brand,google,omg,Web,...,0,0,1,0,2014,1,1,2014,1,1
3,ncf87guaf0,OTHER,33.590916,basic,0,en,direct,direct,linked,Web,...,0,2,0,0,2014,1,1,2014,1,1
4,4rvqpxoh3h,OTHER,33.001512,basic,25,en,direct,direct,untracked,iOS,...,1,0,0,0,2014,1,1,2014,1,1


In [145]:
features.fillna(-1,inplace=True)

In [146]:
features

,id,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,...,d_unavailabilities,d_update,d_verify,d_view,dac_year,dac_month,dac_day,tfc_year,tfc_month,tfc_day
0,d1mm9tcy42,MALE,62.000000,basic,0,en,sem-non-brand,google,omg,Web,...,0,0,0,0,2014,1,1,2014,1,1
1,yo8nz8bqcq,OTHER,37.861191,basic,0,en,direct,direct,untracked,Web,...,0,0,0,0,2014,1,1,2014,1,1
2,4grx6yxeby,OTHER,37.861191,basic,0,en,sem-brand,google,omg,Web,...,0,0,1,0,2014,1,1,2014,1,1
3,ncf87guaf0,OTHER,33.590916,basic,0,en,direct,direct,linked,Web,...,0,2,0,0,2014,1,1,2014,1,1
4,4rvqpxoh3h,OTHER,33.001512,basic,25,en,direct,direct,untracked,iOS,...,1,0,0,0,2014,1,1,2014,1,1
5,c8mfesvkv0,OTHER,36.124620,basic,0,en,direct,direct,linked,Web,...,0,0,0,0,2014,1,1,2014,1,1
6,xwxei6hdk4,FEMALE,32.000000,facebook,0,en,seo,google,linked,Web,...,0,0,0,0,2014,1,1,2014,1,1
7,5f45ro5uzk,OTHER,33.590916,basic,0,en,direct,direct,untracked,Web,...,0,1,0,0,2014,1,1,2014,1,1
8,ro2stddszp,OTHER,19.000000,basic,0,en,sem-brand,google,untracked,Web,...,0,0,0,0,2014,1,1,2014,1,1
9,qtw88d9pbl,MALE,25.000000,basic,0,en,direct,direct,untracked,Web,...,0,0,1,0,2014,1,1,2014,1,1


In [147]:
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider','first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    dummies = pd.get_dummies(features[f]).rename(columns=lambda x: f+'_'+str(x))
    features = pd.concat((features, dummies), axis=1)
    features.drop([f], axis=1,inplace=True)

In [148]:
labels.shape

(122631,)

In [149]:
training_split = features[features.id.isin(train_id)]
testing_split = features[features.id.isin(test_id)].reset_index(drop=True)

In [150]:
training_split

,id,age,secs_elapsed,d_account,d_active,d_agree_terms_check,d_ajax_check_dates,d_ajax_get_referrals_amt,d_ajax_image_upload,d_ajax_lwlb_contact,...,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,d1mm9tcy42,62.000000,3427529,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,yo8nz8bqcq,37.861191,207842,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4grx6yxeby,37.861191,1135444,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ncf87guaf0,33.590916,3755100,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4rvqpxoh3h,33.001512,2555,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,c8mfesvkv0,36.124620,1380288,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,xwxei6hdk4,32.000000,49673,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,5f45ro5uzk,33.590916,605413,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,ro2stddszp,19.000000,1284401,0,0,0,2,0,0,2,...,0,0,0,0,0,0,0,0,0,0
9,qtw88d9pbl,25.000000,2868205,0,0,0,16,0,0,7,...,0,0,0,0,0,0,0,0,0,0


In [151]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels) 

In [152]:

training_split.drop(['id'],inplace=True,axis=1)
testing_id = testing_split['id']
testing_split.drop(['id'],inplace=True,axis=1)
# training_split.drop(['country_destination'],inplace=True,axis=1)
# testing_split.drop(['country_destination'],inplace=True,axis=1)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [53]:
model1 = xgb.XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5)

model2 = xgb.XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=100,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5)

model3 = xgb.XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=50,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5)
# # xgb.XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
# #                     objective='multi:softprob', subsample=0.5, colsample_bytree=0.5)
# # model = xgb.XGBClassifier(objective='multi:softprob')

# # random_forest = RandomForestClassifier(n_estimators = 100, max_depth =6)

print "cross val score: {}".format(cross_val_score(model1, training_split, y, cv=3).mean())
print "cross val score: {}".format(cross_val_score(model2, training_split, y, cv=3).mean())
print "cross val score: {}".format(cross_val_score(model3, training_split, y, cv=3).mean())

cross val score: 0.59270768048
cross val score: 0.558880002363
cross val score: 0.563236174133


In [159]:
model = xgb.XGBClassifier(max_depth=4, learning_rate=0.3, n_estimators=200,
                    objective='multi:softprob', subsample=0.7, colsample_bytree=0.7)
X_train, X_test, Y_train, Y_test = train_test_split(training_split, y, test_size=0.2, random_state=1)
model.fit(X_train,Y_train)
print classification_report(Y_test,model.predict(X_test))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        53
          1       0.00      0.00      0.00       151
          2       0.00      0.00      0.00       104
          3       0.00      0.00      0.00       241
          4       0.11      0.00      0.00       532
          5       0.00      0.00      0.00       248
          6       0.40      0.01      0.01       296
          7       0.73      0.87      0.79     14533
          8       0.00      0.00      0.00        85
          9       0.00      0.00      0.00        27
         10       0.53      0.54      0.53      7028
         11       0.08      0.00      0.00      1229

avg / total       0.60      0.67      0.62     24527



In [154]:
model = xgb.XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200,
                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6)
# X_train, X_test, Y_train, Y_test = train_test_split(training_split, y, test_size=0.2, random_state=1)
model.fit(X_train,Y_train)
print classification_report(Y_test,model.predict(X_test))

             precision    recall  f1-score   support

          0       0.78      0.13      0.23        53
          1       0.69      0.06      0.11       151
          2       0.71      0.21      0.33       104
          3       0.54      0.06      0.10       241
          4       0.49      0.07      0.12       532
          5       0.64      0.08      0.15       248
          6       0.67      0.07      0.13       296
          7       0.73      0.87      0.79     14533
          8       0.79      0.13      0.22        85
          9       1.00      0.04      0.07        27
         10       0.54      0.54      0.54      7028
         11       0.48      0.06      0.11      1229

avg / total       0.66      0.68      0.64     24527



In [155]:
from operator import itemgetter
def report(grid_scores, n_top=3):
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Model with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
              score.mean_validation_score,
              np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")


In [158]:
# use a full grid over all parameters
from time import time
model = RandomForestClassifier()
param_grid = {"max_depth": np.arange(19,25), "n_estimators": [120,140]}

# param_grid = {"max_depth": [3, None],
#               "max_features": [1, 3, 10],
#               "min_samples_split": [1, 3, 10],
#               "min_samples_leaf": [1, 3, 10],
#               "bootstrap": [True, False],
#               "criterion": ["gini", "entropy"]}



# run grid search
gs = grid_search.GridSearchCV(model, param_grid=param_grid)
start = time()
gs.fit(training_split, y,verbose=True)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(gs.grid_scores_)))

report(gs.grid_scores_)

TypeError: fit() got an unexpected keyword argument 'verbose'

In [54]:
gs.grid_scores_

[mean: 0.67281, std: 0.01021, params: {'n_estimators': 100, 'max_depth': 14},
 mean: 0.67567, std: 0.00970, params: {'n_estimators': 120, 'max_depth': 14},
 mean: 0.67572, std: 0.01032, params: {'n_estimators': 100, 'max_depth': 15},
 mean: 0.67467, std: 0.01042, params: {'n_estimators': 120, 'max_depth': 15},
 mean: 0.67590, std: 0.00792, params: {'n_estimators': 100, 'max_depth': 16},
 mean: 0.67610, std: 0.00933, params: {'n_estimators': 120, 'max_depth': 16},
 mean: 0.67590, std: 0.00813, params: {'n_estimators': 100, 'max_depth': 17},
 mean: 0.67643, std: 0.00936, params: {'n_estimators': 120, 'max_depth': 17},
 mean: 0.67749, std: 0.00911, params: {'n_estimators': 100, 'max_depth': 18},
 mean: 0.67692, std: 0.01028, params: {'n_estimators': 120, 'max_depth': 18},
 mean: 0.67759, std: 0.00861, params: {'n_estimators': 100, 'max_depth': 19},
 mean: 0.67816, std: 0.00875, params: {'n_estimators': 120, 'max_depth': 19}]

In [160]:
model = xgb.XGBClassifier(max_depth=10, learning_rate=0.3, n_estimators=200,
                    objective='multi:softprob', subsample=0.6, colsample_bytree=0.6)
model.fit(training_split, y)

XGBClassifier(base_score=0.5, colsample_bytree=0.6, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=10,
       min_child_weight=1, n_estimators=200, nthread=-1,
       objective='multi:softprob', seed=0, silent=True, subsample=0.6)

In [163]:
random_forest = RandomForestClassifier(n_estimators=200, max_depth=10,n_jobs=-1)
random_forest.fit(training_split,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [111]:
model = xgb.XGBClassifier(max_depth=3, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5)
model.fit(training_split,y)

XGBClassifier(base_score=0.5, colsample_bytree=0.5, gamma=0,
       learning_rate=0.3, max_delta_step=0, max_depth=3,
       min_child_weight=1, n_estimators=25, nthread=-1,
       objective='multi:softprob', seed=0, silent=True, subsample=0.5)

In [164]:
y_pred = random_forest.predict_proba(testing_split) 

In [58]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer


def dcg_score(y_true, y_score, k=5):
    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions)
    """
    lb = LabelBinarizer()
    T = lb.fit_transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)

# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True)

In [ ]:
# ndcg_scorer([1,2,3,4,5,6,7,8,9,10,11],y_pred,random_forest)

ndcg_scorer(model, training_split, y)

In [165]:
#Taking the 5 classes with highest probabilities
ids = []  #list of ids
cts = []  #list of countries
for i in range(len(testing_id)):
    idx = testing_id[i]
    ids += [idx] * 5
    cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

#Generate submission
sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
sub.to_csv('sub.csv',index=False)

In [ ]:
xgb <- xgboost(data = data.matrix(X[,-1]), 
               label = y, 
               eta = 0.03,
               max_depth = 30, 
               nround=10, 
               subsample = 0.6,
               colsample_bytree = 0.6,
               seed = 0,
               eval_metric = ndcg5,
               objective = "multi:softprob",
               num_class = 13,
               nthread = 8
)

In [204]:
tr = training_split[training_split.action>0]

In [206]:
tr.reset_index(inplace=True,drop=True)
tr.head()


,age,country_destination,id,action,secs_elapsed,dac_year,dac_month,dac_day,tfc_year,tfc_month,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,62,other,d1mm9tcy42,127,3427529,2014,1,1,2014,1,...,0,0,0,0,0,0,0,0,0,0
1,-1,NDF,yo8nz8bqcq,9,207842,2014,1,1,2014,1,...,0,0,0,0,0,0,0,0,0,0
2,-1,NDF,4grx6yxeby,16,1135444,2014,1,1,2014,1,...,0,0,0,0,0,0,0,0,0,0
3,-1,NDF,ncf87guaf0,152,3755100,2014,1,1,2014,1,...,0,0,0,0,0,0,0,0,0,0
4,-1,GB,4rvqpxoh3h,8,2555,2014,1,1,2014,1,...,0,0,0,0,0,0,0,0,0,0


In [221]:
tr.shape[0]

73815

In [215]:
not_tr = training_split[training_split.action<=0]
not_tr.shape[0]

139636

In [218]:
sample_index = np.random.randint(tr.shape[0],size = (not_tr.shape[0]*2))

In [110]:
sample_index = np.random.randint(training_split.shape[0],size = tr_tr.shape[0])

In [219]:
len(sample_index)

279272

In [222]:
tr_tr = pd.concat([not_tr,tr.iloc[sample_index]])

In [223]:
print(tr_tr[tr_tr.action >=0].shape[0])
print(tr_tr.shape[0])

279272
418908


In [229]:
tr_tr.reset_index(inplace=True,drop=True)
labels = tr_tr['country_destination']
tr_tr.drop('country_destination', inplace=True,axis=1)
tr_tr.drop('id',inplace=True,axis=1)

In [230]:
tr_tr.head()

,age,action,secs_elapsed,dac_year,dac_month,dac_day,tfc_year,tfc_month,tfc_day,gender_FEMALE,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,-1,-1,-1,2010,6,28,2009,3,19,0,...,0,0,0,0,0,0,0,0,0,0
1,38,-1,-1,2011,5,25,2009,5,23,0,...,0,0,0,0,0,0,0,0,0,0
2,56,-1,-1,2010,9,28,2009,6,9,1,...,0,0,0,0,0,0,0,0,0,0
3,42,-1,-1,2011,12,5,2009,10,31,1,...,0,0,0,0,0,0,0,0,0,0
4,41,-1,-1,2010,9,14,2009,12,8,0,...,0,0,0,0,0,0,0,0,0,0


In [232]:
testing_split.reset_index(inplace=True,drop=True)
testing_split.drop('country_destination',inplace=True,axis=1)

/Users/raymondma/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [237]:
testing_split.head()

,age,id,action,secs_elapsed,dac_year,dac_month,dac_day,tfc_year,tfc_month,tfc_day,...,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_UC Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser
0,35,5uwns89zht,8,119187,2014,7,1,2014,7,1,...,0,0,0,0,0,0,0,0,0,0
1,-1,jtl0dijy2j,19,250119,2014,7,1,2014,7,1,...,0,0,0,0,0,0,0,0,0,0
2,-1,xx0ulgorjt,58,975575,2014,7,1,2014,7,1,...,0,0,0,0,0,0,0,0,0,0
3,-1,6c6puo6ix0,11,123001,2014,7,1,2014,7,1,...,0,0,0,0,0,0,0,0,0,0
4,-1,czqhjk3yfe,19,454023,2014,7,1,2014,7,1,...,0,0,0,0,0,0,0,0,0,0


0           NDF
1           NDF
2            US
3         other
4            US
5            US
6            US
7            US
8            US
9            US
10           US
11          NDF
12           FR
13          NDF
14          NDF
15           CA
16          NDF
17           US
18          NDF
19           FR
20           US
21           US
22           US
23          NDF
24          NDF
25           FR
26          NDF
27           US
28           FR
29           US
          ...  
418878       US
418879       US
418880      NDF
418881      NDF
418882      NDF
418883      NDF
418884       US
418885      NDF
418886      NDF
418887      NDF
418888      NDF
418889      NDF
418890      NDF
418891       US
418892       US
418893      NDF
418894      NDF
418895       US
418896       US
418897       US
418898       US
418899       ES
418900      NDF
418901       US
418902      NDF
418903      NDF
418904       US
418905       US
418906       IT
418907      NDF
Name: country_destinatio

In [242]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(labels) 